https://bit.ly/FEML-7-FEAT

## Работа с переменными

<p style="align: center;"><img align=center src="https://netology.ru/backend/tilda/images/tild3439-3364-4535-b334-656263633534__main.svg"  width=900></p>


In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
 
from matplotlib import pyplot as plt
 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [58]:
data = pd.read_csv('new_data.csv')
data.head(5)

,Unnamed: 0.1,Unnamed: 0,address,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zindexvalue,zipcode,zpid
0,0,2,Address: 1160 Mission Street #2007,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,Condominium,2007.0,1167508.0,"975,700",94103.0,83152781.0
1,1,5,Address: 260 King Street #475,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,Condominium,2004.0,823719.0,"975,700",94107.0,69819817.0
2,2,7,Address: 560 Missouri Street #B,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6,Condominium,2003.0,1708594.0,"1,277,600",94107.0,64972847.0
3,3,9,Address: 350 Missouri Street,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10,SingleFamily,1927.0,2411236.0,"1,277,600",94107.0,15149489.0
4,4,11,Address: 3658 Folsom Street,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4,SingleFamily,1900.0,1918539.0,"1,248,000",94110.0,15161978.0


In [59]:
data.shape

(11330, 19)

In [60]:
data.isna().sum()

Unnamed: 0.1     0
Unnamed: 0       0
address          0
z_address        0
bathrooms        0
bedrooms         0
finishedsqft     0
lastsolddate     0
lastsoldprice    0
latitude         0
longitude        0
neighborhood     0
totalrooms       0
usecode          0
yearbuilt        0
zestimate        0
zindexvalue      0
zipcode          0
zpid             0
dtype: int64

Будем использовать линейную регрессию для тестирования качества данных

In [61]:
from sklearn.preprocessing import StandardScaler


def get_score(X,y, random_seed=42, model=None):
  if model is None:
      model = LinearRegression()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed) 

  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  model.fit(X_train, y_train)
  return model.score(X_test, y_test)

In [62]:
for_drop = ['Unnamed: 0', 'Unnamed: 0.1', 'address', 'zindexvalue', 'zpid']
data_processed = data.drop(columns=for_drop)

In [63]:
!pip install category_encoders

In [64]:
from sklearn.preprocessing import LabelEncoder
from category_encoders.count import CountEncoder


def get_one_hot(X, cols):
    for each in cols:
        dummies = pd.get_dummies(X[each], prefix=each)
        X = pd.concat([X, dummies], axis=1)
    return X

def get_label(X, cols):
    X = X.copy()
    for each in cols:
        le = LabelEncoder()
        labels = le.fit_transform(X[each])
        X[each] = labels
    return X

def get_count(X, cols):
    X = X.copy()
    
    for col in cols:
        X[col] = X[col].astype('str')

    ce = CountEncoder(handle_unknown=-1)
    ce.fit(X[cols])
    X[cols] = ce.transform(X[cols])
    return X

In [65]:
data_processed.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zipcode
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,Condominium,2007.0,1167508.0,94103.0
1,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,Condominium,2004.0,823719.0,94107.0
2,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6,Condominium,2003.0,1708594.0,94107.0
3,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10,SingleFamily,1927.0,2411236.0,94107.0
4,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4,SingleFamily,1900.0,1918539.0,94110.0


In [66]:
columns = [
    'bathrooms',
    'bedrooms',
    'finishedsqft',
    'latitude',
    'longitude',
    'totalrooms'
]

In [67]:
get_score(data_processed[columns], data_processed['zestimate'])

0.6462356869965428

## Обработка категорий

In [68]:
exclude_columns = ['z_address', 'lastsoldprice', 'zestimate', 'lastsolddate', 'neighborhood', 'usecode']

### usecode

In [69]:
data_processed['usecode'].value_counts()

usecode
SingleFamily        5785
Condominium         4782
MultiFamily2To4      484
Duplex               146
Townhouse             66
other                 40
Miscellaneous         17
Apartment              3
Cooperative            3
Mobile                 2
MultiFamily5Plus       2
Name: count, dtype: int64

In [70]:
tmp = get_one_hot(data_processed, cols=['usecode'])
tmp.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,...,usecode_Condominium,usecode_Cooperative,usecode_Duplex,usecode_Miscellaneous,usecode_Mobile,usecode_MultiFamily2To4,usecode_MultiFamily5Plus,usecode_SingleFamily,usecode_Townhouse,usecode_other
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,...,True,False,False,False,False,False,False,False,False,False
1,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,...,True,False,False,False,False,False,False,False,False,False
2,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6,...,True,False,False,False,False,False,False,False,False,False
3,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10,...,False,False,False,False,False,False,False,True,False,False
4,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4,...,False,False,False,False,False,False,False,True,False,False


In [71]:
get_score(tmp.drop(columns=exclude_columns), data_processed['zestimate'])

0.6751160687570961

In [72]:
tmp = get_count(data_processed, cols=['usecode'])
tmp.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,usecode,yearbuilt,zestimate,zipcode
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,4782,2007.0,1167508.0,94103.0
1,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,4782,2004.0,823719.0,94107.0
2,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6,4782,2003.0,1708594.0,94107.0
3,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10,5785,1927.0,2411236.0,94107.0
4,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4,5785,1900.0,1918539.0,94110.0


In [73]:
get_score(tmp.drop(columns=exclude_columns), data_processed['zestimate'])

0.6552571011724768

In [74]:
data_processed = get_one_hot(data_processed, cols=['usecode'])

### neighborhood

In [75]:
tmp = get_one_hot(data_processed, cols=['neighborhood'])
tmp.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,...,neighborhood_Sunnyside,neighborhood_Telegraph Hill,neighborhood_Twin Peaks,neighborhood_Van Ness - Civic Center,neighborhood_Visitacion Valley,neighborhood_West Portal,neighborhood_Western Addition,neighborhood_Westwood Highlands,neighborhood_Westwood Park,neighborhood_Yerba Buena
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,South of Market,4,...,False,False,False,False,False,False,False,False,False,False
1,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,South of Market,3,...,False,False,False,False,False,False,False,False,False,False
2,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,Potrero Hill,6,...,False,False,False,False,False,False,False,False,False,False
3,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,Potrero Hill,10,...,False,False,False,False,False,False,False,False,False,False
4,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,Bernal Heights,4,...,False,False,False,False,False,False,False,False,False,False


In [76]:
get_score(tmp.drop(columns=exclude_columns), data_processed['zestimate'])

0.7500797357821893

In [77]:
data_processed['neighborhood'].value_counts()

neighborhood
Mission             540
Bernal Heights      525
South of Market     524
South Beach         461
Pacific Heights     439
                   ... 
North Beach          31
Lakeside             29
Little Hollywood     12
North Waterfront      8
Daly City             3
Name: count, Length: 71, dtype: int64

In [78]:
tmp = get_count(data_processed, cols=['neighborhood'])
tmp.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,...,usecode_Condominium,usecode_Cooperative,usecode_Duplex,usecode_Miscellaneous,usecode_Mobile,usecode_MultiFamily2To4,usecode_MultiFamily5Plus,usecode_SingleFamily,usecode_Townhouse,usecode_other
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,02/17/2016,1300000.0,37.778705,-122.412635,524,4,...,True,False,False,False,False,False,False,False,False,False
1,260 King St UNIT 475,1.0,1.0,903.0,02/17/2016,750000.0,37.777641,-122.393417,524,3,...,True,False,False,False,False,False,False,False,False,False
2,560 Missouri St # B,4.0,3.0,1425.0,02/17/2016,1495000.0,37.759198,-122.396516,312,6,...,True,False,False,False,False,False,False,False,False,False
3,350 Missouri St,3.0,3.0,2231.0,02/17/2016,2700000.0,37.761886,-122.396769,312,10,...,False,False,False,False,False,False,False,True,False,False
4,3658 Folsom St,3.0,3.0,1300.0,02/17/2016,1530000.0,37.740795,-122.413453,525,4,...,False,False,False,False,False,False,False,True,False,False


In [79]:
get_score(tmp.drop(columns=exclude_columns), data_processed['zestimate'])

0.6751160687570961

In [80]:
data_processed = get_one_hot(data_processed, cols=['neighborhood'])

### lastsolddate

In [81]:
data_processed['lastsolddate'].head()

0    02/17/2016
1    02/17/2016
2    02/17/2016
3    02/17/2016
4    02/17/2016
Name: lastsolddate, dtype: object

In [82]:
data_processed['lastsolddate'] = pd.to_datetime(data_processed['lastsolddate'])

In [83]:
data_processed['lastsolddate'].head()

0   2016-02-17
1   2016-02-17
2   2016-02-17
3   2016-02-17
4   2016-02-17
Name: lastsolddate, dtype: datetime64[ns]

In [84]:
data_processed['lastsoldmonth'] = data_processed['lastsolddate'].dt.month
data_processed['lastsoldmonth']

0        2
1        2
2        2
3        2
4        2
        ..
11325    2
11326    2
11327    2
11328    2
11329    2
Name: lastsoldmonth, Length: 11330, dtype: int32

In [85]:
data_processed['lastsolddate'] = [t.timestamp() for t in data_processed['lastsolddate']]
data_processed['lastsolddate']

0        1.455667e+09
1        1.455667e+09
2        1.455667e+09
3        1.455667e+09
4        1.455667e+09
             ...     
11325    1.454458e+09
11326    1.454371e+09
11327    1.454371e+09
11328    1.454371e+09
11329    1.454285e+09
Name: lastsolddate, Length: 11330, dtype: float64

In [86]:
# exclude_columns = ['z_address', 'lastsoldprice', 'zestimate', 'lastsolddate', 'neighborhood', 'usecode']
exclude_columns = ['z_address', 'lastsoldprice', 'zestimate', 'neighborhood', 'usecode']

In [87]:
get_score(data_processed.drop(columns=exclude_columns), data_processed['zestimate'])

0.7503779640995267

In [88]:
tmp = get_one_hot(data_processed, cols=['lastsoldmonth']).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,lastsoldmonth_3,lastsoldmonth_4,lastsoldmonth_5,lastsoldmonth_6,lastsoldmonth_7,lastsoldmonth_8,lastsoldmonth_9,lastsoldmonth_10,lastsoldmonth_11,lastsoldmonth_12
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,94103.0,False,...,False,False,False,False,False,False,False,False,False,False
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,94110.0,False,...,False,False,False,False,False,False,False,False,False,False


In [89]:
get_score(tmp, data_processed['zestimate'])

0.7504405975771404

In [90]:
tmp = get_one_hot(data_processed[['lastsoldmonth']], cols=['lastsoldmonth'])
tmp.head()

,lastsoldmonth,lastsoldmonth_1,lastsoldmonth_2,lastsoldmonth_3,lastsoldmonth_4,lastsoldmonth_5,lastsoldmonth_6,lastsoldmonth_7,lastsoldmonth_8,lastsoldmonth_9,lastsoldmonth_10,lastsoldmonth_11,lastsoldmonth_12
0,2,False,True,False,False,False,False,False,False,False,False,False,False
1,2,False,True,False,False,False,False,False,False,False,False,False,False
2,2,False,True,False,False,False,False,False,False,False,False,False,False
3,2,False,True,False,False,False,False,False,False,False,False,False,False
4,2,False,True,False,False,False,False,False,False,False,False,False,False


In [91]:
get_score(tmp, data_processed['zestimate'])

0.0035635462867416035

In [92]:
data_processed['lastsoldmonth'].value_counts()

lastsoldmonth
10    1433
8     1165
11    1088
7     1075
12     988
9      983
4      872
3      863
2      817
1      759
5      682
6      605
Name: count, dtype: int64

In [93]:
tmp = get_count(data_processed, cols=['lastsoldmonth']).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,neighborhood_Telegraph Hill,neighborhood_Twin Peaks,neighborhood_Van Ness - Civic Center,neighborhood_Visitacion Valley,neighborhood_West Portal,neighborhood_Western Addition,neighborhood_Westwood Highlands,neighborhood_Westwood Park,neighborhood_Yerba Buena,lastsoldmonth
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,94103.0,False,...,False,False,False,False,False,False,False,False,False,817
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,817
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,817
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,817
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,94110.0,False,...,False,False,False,False,False,False,False,False,False,817


In [94]:
get_score(tmp, data_processed['zestimate'])

0.7507036036196328

In [95]:
data_processed = get_count(data_processed, cols=['lastsoldmonth'])

### zipcode

In [96]:
tmp = get_one_hot(data_processed, cols=['zipcode']).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,zipcode_94121.0,zipcode_94122.0,zipcode_94123.0,zipcode_94124.0,zipcode_94127.0,zipcode_94131.0,zipcode_94132.0,zipcode_94133.0,zipcode_94134.0,zipcode_94158.0
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,94103.0,False,...,False,False,False,False,False,False,False,False,False,False
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,94107.0,False,...,False,False,False,False,False,False,False,False,False,False
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,94110.0,False,...,False,False,False,False,False,False,False,False,False,False


In [97]:
get_score(tmp, data_processed['zestimate'])

0.7562573505100021

In [98]:
tmp = get_label(data_processed, cols=['zipcode']).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,neighborhood_Telegraph Hill,neighborhood_Twin Peaks,neighborhood_Van Ness - Civic Center,neighborhood_Visitacion Valley,neighborhood_West Portal,neighborhood_Western Addition,neighborhood_Westwood Highlands,neighborhood_Westwood Park,neighborhood_Yerba Buena,lastsoldmonth
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,1,False,...,False,False,False,False,False,False,False,False,False,817
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,4,False,...,False,False,False,False,False,False,False,False,False,817
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,4,False,...,False,False,False,False,False,False,False,False,False,817
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,4,False,...,False,False,False,False,False,False,False,False,False,817
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,7,False,...,False,False,False,False,False,False,False,False,False,817


In [99]:
get_score(tmp, data_processed['zestimate'])

0.7512019855682772

In [100]:
tmp = get_count(data_processed, cols=['zipcode']).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,neighborhood_Telegraph Hill,neighborhood_Twin Peaks,neighborhood_Van Ness - Civic Center,neighborhood_Visitacion Valley,neighborhood_West Portal,neighborhood_Western Addition,neighborhood_Westwood Highlands,neighborhood_Westwood Park,neighborhood_Yerba Buena,lastsoldmonth
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,360,False,...,False,False,False,False,False,False,False,False,False,817
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,857,False,...,False,False,False,False,False,False,False,False,False,817
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,857,False,...,False,False,False,False,False,False,False,False,False,817
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,857,False,...,False,False,False,False,False,False,False,False,False,817
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,935,False,...,False,False,False,False,False,False,False,False,False,817


In [101]:
get_score(tmp, data_processed['zestimate'])

0.7487701270930991

In [102]:
data_processed = get_one_hot(data_processed, cols=['zipcode'])

### z_address

<img src='https://drive.google.com/uc?export=view&id=14nvoZ71VqMfYBY6ff_7_Btx-tVwL_4lB'>

TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова t в пределах отдельного документа.

$$tf = \frac {n_t}{\sum _{k}n_{k}}$$

*TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)*



IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции.

$$idf=\log \frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}$$

*IDF термина а = логарифм(Общее количество документов / Количество документов, в которых встречается термин а)*



Таким образом, мера TF-IDF является произведением двух сомножителей:

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах. 

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=50, stop_words='english')
vectorized = tfidf.fit_transform(data_processed['z_address'])

vectorized

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 17826 stored elements and shape (11330, 50)>

In [104]:
vectorized_df = pd.DataFrame(vectorized.toarray(), columns=[str(i) for i in range(50)])
vectorized_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.207690,0.0,0.0,0.498299,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.167233,0.0,0.0,0.401232,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.247710,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
11326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
11327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
11328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [105]:
tmp = pd.concat([data_processed, vectorized_df], axis=1).drop(columns=exclude_columns)
tmp.head()

,bathrooms,bedrooms,finishedsqft,lastsolddate,latitude,longitude,totalrooms,yearbuilt,zipcode,usecode_Apartment,...,40,41,42,43,44,45,46,47,48,49
0,2.0,2.0,1043.0,1.455667e+09,37.778705,-122.412635,4,2007.0,94103.0,False,...,0.0,0.0,0.0,0.0,0.207690,0.0,0.0,0.498299,0.0,0.0
1,1.0,1.0,903.0,1.455667e+09,37.777641,-122.393417,3,2004.0,94107.0,False,...,0.0,0.0,0.0,0.0,0.167233,0.0,0.0,0.401232,0.0,0.0
2,4.0,3.0,1425.0,1.455667e+09,37.759198,-122.396516,6,2003.0,94107.0,False,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
3,3.0,3.0,2231.0,1.455667e+09,37.761886,-122.396769,10,1927.0,94107.0,False,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0
4,3.0,3.0,1300.0,1.455667e+09,37.740795,-122.413453,4,1900.0,94110.0,False,...,0.0,0.0,0.0,0.0,0.247710,0.0,0.0,0.000000,0.0,0.0


In [106]:
get_score(tmp, data_processed['zestimate'])

0.7571843833351609

In [107]:
data_processed = pd.concat([data_processed, vectorized_df], axis=1)

## Работа с геоданными

In [108]:
from sklearn.cluster import DBSCAN, KMeans

In [109]:
len(data_processed['zipcode'].unique())

25

In [110]:
# kmeans = KMeans(n_clusters=25)
kmeans = KMeans(n_clusters=15)

cluster = kmeans.fit_predict(data_processed[['latitude', 'longitude']])

In [111]:
tmp = data_processed.copy()
tmp['cluster'] = cluster
tmp.head()

,z_address,bathrooms,bedrooms,finishedsqft,lastsolddate,lastsoldprice,latitude,longitude,neighborhood,totalrooms,...,41,42,43,44,45,46,47,48,49,cluster
0,1160 Mission St UNIT 2007,2.0,2.0,1043.0,1.455667e+09,1300000.0,37.778705,-122.412635,South of Market,4,...,0.0,0.0,0.0,0.207690,0.0,0.0,0.498299,0.0,0.0,9
1,260 King St UNIT 475,1.0,1.0,903.0,1.455667e+09,750000.0,37.777641,-122.393417,South of Market,3,...,0.0,0.0,0.0,0.167233,0.0,0.0,0.401232,0.0,0.0,5
2,560 Missouri St # B,4.0,3.0,1425.0,1.455667e+09,1495000.0,37.759198,-122.396516,Potrero Hill,6,...,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,7
3,350 Missouri St,3.0,3.0,2231.0,1.455667e+09,2700000.0,37.761886,-122.396769,Potrero Hill,10,...,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,7
4,3658 Folsom St,3.0,3.0,1300.0,1.455667e+09,1530000.0,37.740795,-122.413453,Bernal Heights,4,...,0.0,0.0,0.0,0.247710,0.0,0.0,0.000000,0.0,0.0,8


In [112]:
get_score(tmp.drop(columns=exclude_columns), data_processed['zestimate'])

0.7586926910777572

In [113]:
data_processed = tmp.copy()

In [114]:
!pip install reverse_geocoder

     ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
     ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
     ---------------------------------------- 2.2/2.2 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for reverse_geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268080 sha256=c1de1220b9dfe191def27a50fcd4603169a95ac7b8477d259d537df893d078d6
  Stored in directory: c:\users\lipatov\appdata\local\pip\cache\wheels\11\e1\67\6e47f0ad41ea1843d37e1fbe79c6074744a1f4aace641cf800
Successfully built reverse_geocoder


In [115]:
import reverse_geocoder as revgc  

revgc.search((data_processed.iloc[10].latitude, data_processed.iloc[10].longitude))   

Loading formatted geocoded file...


[{'lat': '37.68077',
  'lon': '-122.39997',
  'name': 'Brisbane',
  'admin1': 'California',
  'admin2': 'San Mateo County',
  'cc': 'US'}]

In [116]:
!pip install folium

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached charset_normalizer-3.4.0-cp312-cp312-win_amd64.whl (102 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)


In [117]:
import folium
this_map = folium.Map(prefer_canvas=True)

def plotDot(point, color):
    
    folium.CircleMarker(
        location=[point.latitude, point.longitude],
        radius=2,
        weight=5,
        color=color,
        popup=point.zestimate
    ).add_to(this_map)
    
data_processed.iloc[:2000].apply(plotDot, axis=1, color='#3388FF')

this_map.fit_bounds(this_map.get_bounds())

this_map